In [75]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Load the training data

In [76]:
from helpers import *

y, tX, ids = load_csv_data('Data/train.csv')
_, tX_test, ids_test = load_csv_data('Data/test.csv')

# Preprocessing

### Umbalanced Dataset
IDEA:We could use Undersampling\Oversampling alghoritms

In [77]:
higgs = np.count_nonzero(y==1)
print(f'From {y.shape[0]} training examples, {higgs} are 1, i.e. the {higgs/y.shape[0]} %')

From 250000 training examples, 85667 are 1, i.e. the 0.342668 %


### Manage missing values
Play with parameters

Riccardo: create a single function for train e test

In [78]:
tX = missing_values(tX)
tX_test = missing_values(tX_test)

### Standadization

In [79]:
tX = standardize(tX)
tX_test = standardize(tX_test)

### Other Ideas
PCA, correlation analysis (scatterplot, VIF, ...), manage the 0s in the last feature, new features

Outlayer analysis, leverages, cook's metric ...

# Methods

In [89]:
from methods import *

## 1. Least Squares with no Gradient Descent

In [84]:
loss, weights = least_squares(y, tX)

## 2. Least Squares with Gradient Descent

In [85]:
loss, weights = gradient_descent(y,tX,1000,0.01)

Gradient Descent(0/999): loss=15.906261952413804
Gradient Descent(100/999): loss=0.724647570179371
Gradient Descent(200/999): loss=0.5167426193184895
Gradient Descent(300/999): loss=0.4551667927727656
Gradient Descent(400/999): loss=0.4291397848451059
Gradient Descent(500/999): loss=0.415762519007584
Gradient Descent(600/999): loss=0.40794989013658345
Gradient Descent(700/999): loss=0.4029982677944452
Gradient Descent(800/999): loss=0.399684129707814
Gradient Descent(900/999): loss=0.397375846166907


## 5. Logistic Regression with Stochastic Gradient Descent

In [94]:
loss, weights = logistic_regression_gradient_descent(y, tX)

282341.0273518375
nan
nan
nan
nan
nan
nan
nan

C:\Users\Raphael\higgs_boson_classification\methods.py:49: RuntimeWarning: overflow encountered in exp
  return np.exp(t)/(1+np.exp(t))
C:\Users\Raphael\higgs_boson_classification\methods.py:49: RuntimeWarning: invalid value encountered in true_divide
  return np.exp(t)/(1+np.exp(t))
C:\Users\Raphael\higgs_boson_classification\methods.py:54: RuntimeWarning: divide by zero encountered in log
  loss = -y.T.dot(np.log(pred)) + (1 - y).T.dot(np.log(1 - pred))



nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan


## 7. Other Ideas
SVM, DecisionTree, RandomFrest, ...

# Cross Validation

# Prediction

#### Generate predictions and save ouput in csv format for submission

In [48]:
OUTPUT_PATH = 'data/FIRSTsubmission.csv' 
y_pred = predict_labels(weights, tX_test)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)

ATTENTION: we are prediction too many 1

In [49]:
higgs = np.count_nonzero(y_pred==1)
print(f'From {y_pred.shape[0]} test examples, {higgs} are 1, i.e. the {higgs/y_pred.shape[0]} %')

From 568238 test examples, 267427 are 1, i.e. the 0.4706249846015226 %
